In [8]:
desired_target, target = "quantinuum.sim.h1-1sc", None

In [11]:
from azure.quantum.cirq import AzureQuantumService
from cirq.vis import plot_state_histogram
import cirq

service = AzureQuantumService(
            resource_id = "/subscriptions/a5abf7fd-dcef-47af-8385-2fdf76385f21/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/qvc-2",
            location = "eastus")

for tgt in service.targets():
   if tgt.current_availability == "Available":
      if tgt.name == desired_target:
         print("-> " + tgt.name)
         target = tgt
      else:
         print(tgt.name)

from functools import wraps

def subcircuit(circuit_fn):
   @wraps(circuit_fn)
   def inner(*args, **kwargs):
      circ = circuit_fn(*args, **kwargs)
      return cirq.CircuitOperation(circ.freeze())

ionq.qpu.aria-1
ionq.simulator
quantinuum.qpu.h1-1
-> quantinuum.sim.h1-1sc
quantinuum.sim.h1-1e


In [ ]:
# First things first, we need to be able to create the Generalized W state
# The most efficient way that I can think of to do this is as follows:
@subcircuit
def generalized_w(lg_n: int):
    # TODO